You may find this series of notebooks at [databricks-solutions/realtime-rag-agents-databricks-youcom](https://github.com/databricks-solutions/realtime-rag-agents-databricks-youcom). For more information about this solution accelerator, visit the [blog post](https://you.com/articles/unlocking-real-time-intelligence-for-ai-agents-with-you.com-and-databricks).

In [0]:
dbutils.library.restartPython()

In [0]:
import pandas as pd
import pyspark.sql.functions as F
import io
import yaml

import mlflow

import mlflow.genai.datasets

import requests

from agent import AGENT

from mlflow.genai.scorers import (
    RetrievalGroundedness,
    RelevanceToQuery,
    Safety,
    Guidelines,
    Correctness
)

# Create Agent

1. Use the preivous configurations to stream responses from the defined agent.

2. Create a seed dataset that will be evaluated in `04-evaluate-agent`


## 0. Load Configurations from `01-agent-configs`

In [ ]:
# Set up widgets to accept job parameters
dbutils.widgets.text("catalog", "main", "UC Catalog")
dbutils.widgets.text("schema", "default", "UC Schema")

# Get widget values
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

# If widget values are default, try loading from common.yaml as fallback
if catalog == "main" or schema == "default":
    try:
        import yaml
        cfg = yaml.safe_load(open("common.yaml"))
        catalog = cfg.get("catalog", catalog)
        schema = cfg.get("schema", schema)
    except FileNotFoundError:
        pass  # No fallback file, use widget values

uc_schema = f"{catalog}.{schema}"
evaluation_dataset_table_name = "test_evaluation_set"
evaluation_dataset_path = f"{catalog}.{schema}.{evaluation_dataset_table_name}"

# Set environment variables for agent.py to use when imported
import os
os.environ["AGENT_CATALOG"] = catalog
os.environ["AGENT_SCHEMA"] = schema

print(f"Configuration loaded:")
print(f"  Catalog: {catalog}")
print(f"  Schema: {schema}")
print(f"  Evaluation Dataset: {evaluation_dataset_path}")

In [0]:
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
experiment_name = notebook_path + "_exp"

mlflow.set_tracking_uri("databricks")
mlflow.set_experiment(experiment_name)

In [0]:
def create_message_payload(question):
  return {"messages": [{"role": "user", "content": question}]}

## 1. Stream Agent's Response

In [ ]:
question = "What is the current status of United Airlines flight UA2749?"
message = create_message_payload(question)

resp = AGENT.predict(message)
print(resp)

In [0]:
# stream the Assistant's response
for chunk in AGENT.predict_stream(message):
  if chunk.delta.role == "assistant":
    print(chunk.delta.content)

## 2. Seed Examples for Evaluation Dataset

Use the AGENT to predict a couple example questions. Then use the Traces to seed an Evaluation Dataset.

In [ ]:
questions = ["What are the current delays at JFK airport?", 
             "What is the status of Delta flight DL1234 from Atlanta to New York?", ]

run_id = None
with mlflow.start_run() as run:
  for q in questions:
    message = create_message_payload(q)
    resp = AGENT.predict(message)

  run_id = run.info.run_id